# Implementation of False Positive Algorithms

In [74]:
import json
import mysql.connector
import random
import numpy as np
import importlib
imported_module = importlib.import_module("sql_commands")
importlib.reload(imported_module)
import sql_commands
from sql_commands import *
#random.seed(2311)

In [4]:
db_names = ['tpch_2pct', 'tpch_4pct', 'tpch_6pct', 'tpch_8pct', 'tpch_10pct']
null_rates = [0.02, 0.04, 0.06, 0.08, 0.1]

In [5]:
mydb, mycursor = sql_commands.connect_to_db("tpch_og")

In [6]:
all_nations_raw = sql_commands.run_query(mycursor, "SELECT N_NAME FROM NATION")
all_nations = []
for nation in all_nations_raw:
    n = nation[0]
    if n is not None:
        all_nations.append(n)

## Query 1

In [82]:
def q1_fps(query1_out,lineitem_cols,query1_cols):
    # Query to get all items from Lineitem table
    lineitem = sql_commands.run_query(mycursor,"SELECT * FROM LINEITEM")
    # Variables needed for comparing lineitem items to query 1 answer
    l_orderkey_index = lineitem_cols.index('L_ORDERKEY')
    #print("l-orderkey-index", l_orderkey_index)
    o_orderkey_index = query1_cols.index('O_ORDERKEY')
    #print("o-orderkey-index", o_orderkey_index)
    l_suppkey_index = lineitem_cols.index('L_SUPPKEY')
    s_suppkey_index = query1_cols.index("S_SUPPKEY")
    l_commitdate_index = lineitem_cols.index("L_COMMITDATE")
    l_receiptdate_index = lineitem_cols.index('L_RECEIPTDATE')
    count = 0
    continue_count = 0
    not_count = 0
    nothing = 0
    matching_count = 0
    for i,t in enumerate(lineitem):
        for j,a in enumerate(query1_out):
            #print("Lineitem orderkey:", t[l_orderkey_index], "query1 answer orderkey:", a[o_orderkey_index])
            if t[l_orderkey_index] == a[o_orderkey_index]:
                matching_count += 1
                x = t[l_suppkey_index]
                if x is not None and x == a[s_suppkey_index]:
                    continue_count += 1
                    continue
                
                d1 = t[l_commitdate_index]
                d2 = t[l_receiptdate_index]
                if (d1 is None) or (d2 is None) or (d2 > d1):
                    count += 1
                else:
                    not_count += 1
            else:
                nothing += 1
    #print(not_count, matching_count) 
    if matching_count != 0:           
        return count/matching_count
    return None

In [83]:
# list of tuples
# (suppkey, orderkey)
q1_nations = ['UNITED KINGDOM', 'ARGENTINA', 'PERU', 'FRANCE', 'BRAZIL']
q1_results = []
print("QUERY 1 False Positives")
for i in range(len(db_names)):
    curr_db_name = db_names[i]
    curr_null_rate = null_rates[i]
    print(f"\nNull Rate: {curr_null_rate}")
    mydb, mycursor = sql_commands.connect_to_db(curr_db_name)
    fp_pcts = []
    for j in range(3):
        nation = q1_nations[j]
        print(f"Nation: {nation}")
        query1_out = sql_commands.run_query1(mycursor,nation)
        lineitem_attributes = sql_commands.get_attribute_names(mycursor,"LINEITEM")
        query1_attributes = ['S_SUPPKEY', 'O_ORDERKEY']
        fp_pct = q1_fps(query1_out, lineitem_attributes, query1_attributes)
        #print(f"{fp_pct*100:.4}% false positives")
        #print(f"Total Number of rows in query: {len(query1_out)}")
        if fp_pct != None:
            fp_pcts.append(fp_pct)

    #print(fp_pcts)
    print(f"Average false positive rate is: {np.mean(fp_pcts)}")
    q1_results.append(np.mean(fp_pcts))
    

QUERY 1 False Positives

Null Rate: 0.02
Nation: UNITED KINGDOM


KeyboardInterrupt: 

In [ ]:
q1_results

[0.09128052742823267,
 0.1506480897382979,
 0.21371327948268773,
 0.26871189086092767,
 0.3253795296924006]

## Query 2

In [26]:
'''
    Iterate through answer tuples ('c_custkey', 'c_nationkey')
    for each c_custkey
        check whether orders.o_custkey is null
        if yes, fp++
    
    '''
def q2_fps(mycursor, query2_attributes, q2_out):
    false_positives = False
    count = 0
    query = "SELECT * FROM ORDERS WHERE O_CUSTKEY IS NULL"
    result = sql_commands.run_query(mycursor,query)
    # if there is a row where o_custkey is null, then false positive can occur
    for row in result:
        false_positives = True
        count += 1
    if len(result) != 0:
        return count/len(result)
    return None

In [35]:
#random.seed(1123)
q2_results = []
for db_name in db_names:
    fps_q2 = []
    for i in range(5):
        q2_nations = tuple(random.sample(all_nations, 7))
        print(q2_nations)
        mydb, mycursor = sql_commands.connect_to_db(db_name)
        q2_out = sql_commands.run_query2(mycursor, q2_nations)
        #print(q2_out[:5], len(q2_out))

        query2_attributes = ['C_CUSTKEY', 'C_NATIONKEY']
        order_attributes = sql_commands.get_attribute_names(mycursor,"ORDERS")
        #print(order_attributes)
        fp_pct_q2 = q2_fps(mycursor,query2_attributes,q2_out)
        #print("FPs", fp_pct_q2)
        fps_q2.append(fp_pct_q2)
    avg_fp = np.mean(fps_q2)
    q2_results.append(avg_fp)
    print(f"Avg FP Percentage for {db_name} is {avg_fp}\n")


('JAPAN', 'MOZAMBIQUE', 'KENYA', 'ALGERIA', 'FRANCE', 'EGYPT', 'INDIA')
('IRAQ', 'KENYA', 'EGYPT', 'VIETNAM', 'ROMANIA', 'UNITED KINGDOM', 'BRAZIL')
('EGYPT', 'JAPAN', 'CHINA', 'ROMANIA', 'VIETNAM', 'MOROCCO', 'MOZAMBIQUE')
('CHINA', 'MOZAMBIQUE', 'BRAZIL', 'ETHIOPIA', 'KENYA', 'MOROCCO', 'ARGENTINA')
('PERU', 'UNITED STATES', 'GERMANY', 'VIETNAM', 'IRAN', 'JAPAN', 'INDIA')
Avg FP Percentage for tpch_2pct is 1.0

('INDIA', 'CANADA', 'VIETNAM', 'ARGENTINA', 'RUSSIA', 'KENYA', 'BRAZIL')
('ETHIOPIA', 'MOROCCO', 'IRAN', 'JAPAN', 'SAUDI ARABIA', 'INDONESIA', 'BRAZIL')
('BRAZIL', 'INDIA', 'JORDAN', 'ETHIOPIA', 'CHINA', 'RUSSIA', 'UNITED STATES')
('CANADA', 'JORDAN', 'UNITED STATES', 'BRAZIL', 'SAUDI ARABIA', 'ROMANIA', 'IRAQ')
('BRAZIL', 'UNITED STATES', 'IRAN', 'JAPAN', 'INDONESIA', 'ALGERIA', 'PERU')
Avg FP Percentage for tpch_4pct is 1.0

('FRANCE', 'EGYPT', 'CANADA', 'IRAQ', 'ROMANIA', 'CHINA', 'UNITED STATES')
('GERMANY', 'INDONESIA', 'JORDAN', 'BRAZIL', 'CHINA', 'VIETNAM', 'IRAN')
('JO

# Query 3

In [115]:
'''For instance, to falsify
an order id k in the answer to query Q3, we look for a tuple
in table lineitem where the value of attribute l_orderkey
is k and the value of l_suppkey is null.'''
def q3_fps(mycursor, q3_out):
    count = 0
    for order_id in q3_out:
        query = f"""SELECT L_ORDERKEY,L_SUPPKEY
                    FROM LINEITEM
                    WHERE L_ORDERKEY = {order_id}
                    AND L_SUPPKEY IS NULL"""
        result = sql_commands.run_query(mycursor,query)
        
        if len(result) != 0:
            #print(order_id, result)
            count += 1
    #print("Number of FPS", count, "/", len(q3_out))
    if len(q3_out) != 0:
        return count/len(q3_out)
    return None

In [116]:
q3_results = []
for db_name in db_names:    
    fp_avg = []
    for j in range(5):
        mydb, mycursor = sql_commands.connect_to_db(db_name)
        q3_out = sql_commands.run_query3(mycursor)
        fp_q3 = q3_fps(mycursor, q3_out)
        fp_avg.append(fp_q3)
    avg = np.mean(fp_avg)
    q3_results.append(avg)
    print(f"Avg fp rate for {db_name} is {avg:.4}")

Avg fp rate for tpch_2pct is 0.9535
Avg fp rate for tpch_4pct is 0.9736
Avg fp rate for tpch_6pct is 0.9863
Avg fp rate for tpch_8pct is 0.9894
Avg fp rate for tpch_10pct is 0.9916


In [ ]:
sql_commands.close_db(mydb, mycursor)